In [8]:
import pandas as pd
import os
from dbHelpers import createEngine
import uuid

In [9]:
engine = createEngine()

In [10]:
path = '../scratch/halloween_data'

In [12]:
df_dict={}
for f in os.listdir(path):
    if '.csv' in f:
        tablename = f.split('.csv')[0]
        df = pd.read_csv(os.path.join(path,f))
        df_dict[f]=df
        df.to_sql(tablename, index=False, schema = 'raw_csv', con = engine, if_exists='replace')

/var/folders/zt/jvxrh15j40sfz4x_sldwpck00000gp/T/ipykernel_11117/2424709856.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(path,f))


In [ ]:
practitioner_df = df_dict['practitioner.csv']
#note: we can do this because each practitioner only appears once in this table
practitioner_df[id] = [uuid.uuid4() for i in practitioner_df.index]
practitioner_df.set_index('id', in_place=True)
practitioner_df.rename(columns = {'gender_code': 'sex', 'name_prefix': 'prefix', 'name_suffix': 'suffix'}, inplace=True)


In [ ]:
organization_df = df_dict['organization.csv']
organization_df['is_primary'] = True
organization_df.rename(columns={'id':'old_org_id', 'parent_id':'old_parent_id'}, inplace=True)
organization_df['parent_id'] = [uuid.uuid4() for i in organization_df.index]
organization_npi_df = df_dict['organization_npi.csv']
organization_npi_df.rename(columns={'organization_id':'old_org_id'}, inplace=True)
organization_npi_df['id'] = [uuid.uuid4() for i in organization_npi_df.index]
merged_organization_df = organization_npi_df.merge(organization_df, on='old_org_id')


In [ ]:
# load individual
practitioner_df[['id', 'sex']].to_sql('individual', schema = 'npd', con = engine)

practitioner_df.rename(columns={'id':'individual_id'}, inplace=True)

# load individual_to_name
practitioner_df[['individual_id', 'first_name', 'middle_name', 'last_name', 'prefix', 'suffix']].to_sql('individual', schema = 'npd', con = engine)

# load provider
practitioner_df[['npi', 'individual_id']].to_sql('provider', schema='npd', con=engine)

# load organization
merged_organization_df[['id', 'parent_id']].to_sql('organization', schema='npd', con=engine)

merged_organization_df.rename(columns={'id':'organization_id'}, inplace=True)

# load organization_to_name
merged_organization_df[['organization_id', 'name', 'is_primary']].to_sql('organization_to_name', schema = 'npd', con = engine)

# load clinical_organization
merged_organization_df[['organization_id', 'npi']].to_sql('clinical_organization', schema='npd', con = engine)

